In [2]:
import pandas as pd
import numpy as np
import glob
import json

In [16]:
%load_ext autoreload
%autoreload 2

import RDlite.accessor as ac

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from RDlite import all_features

In [17]:
from RDlite import mapping

In [22]:
mapping.keys()

dict_keys(['id', 'company', 'genre', 'keyword', 'language', 'rating', 'country', 'name', 'character', 'gender', 'adult', 'budget', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'release_date', 'revenue', 'runtime', 'status', 'title', 'video', 'vote_average', 'vote_count', 'collection'])

In [5]:
all_features

['company',
 'collection',
 'genre',
 'keyword',
 'rating',
 'language',
 'country',
 'name',
 'character',
 'gender',
 'movie',
 'adult',
 'budget',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'release_date',
 'revenue',
 'runtime',
 'status',
 'video',
 'vote_average',
 'vote_count']

In [6]:
ac.agg(*all_features, peek=True).columns

FileNotFoundError: [Errno 2] No such file or directory: '/home/themaster/workspace/cse314final/Data/collections.csv'

In [10]:
filez = glob.glob("/home/themaster/workspace/cse314final/Data/*")

In [11]:
features = []
for each in filez:
    features += pd.read_csv(each, nrows=1)

ValueError: Unable to coerce to Series, length must be 2: given 0

In [27]:
pd.read_csv(filez[4], nrows=1)

,id,company
0,862,PIXAR ANIMATION STUDIOS


In [23]:
filez

['/home/themaster/workspace/cse314final/Data/genres.csv',
 '/home/themaster/workspace/cse314final/Data/keywords.csv',
 '/home/themaster/workspace/cse314final/Data/ratings.csv',
 '/home/themaster/workspace/cse314final/Data/languages.csv',
 '/home/themaster/workspace/cse314final/Data/companies.csv',
 '/home/themaster/workspace/cse314final/Data/actors.csv',
 '/home/themaster/workspace/cse314final/Data/status_map.csv',
 '/home/themaster/workspace/cse314final/Data/countries.csv',
 '/home/themaster/workspace/cse314final/Data/clean_movie.csv']